
# 06. 최종 앙상블 및 제출 (Final Submission)

본 노트북은 앞서 학습된 **CatBoost, XGBoost, FiBiNet** 3가지 단일 모델의 결과를 결합하여 최종 예측 성능을 극대화합니다.
각 모델의 특성과 검증 점수를 반영한 **가중 앙상블(Weighted Ensemble)** 전략을 사용합니다.

### 🏆 앙상블 전략
1. **모델 가중치 배분**: 
    - **CatBoost**: 40% (범주형 데이터 처리 및 안정성 우수)
    - **XGBoost**: 35% (강력한 패턴 인식 및 과적합 제어)
    - **FiBiNet**: 25% (비선형 상호작용 및 다양성 확보)
2. **Soft Voting**: 각 모델이 예측한 확률값(Probability)의 가중 평균 계산
3. **Submission**: 최종 예측 결과를 `final_submission.csv`로 저장


In [1]:

import pandas as pd
import os

# 작업 디렉토리 설정 (현재 노트북 위치)
os.chdir('/home/konkukstat/python3/workspace/sangmin/CTR-Prediction/models')

# 파일명 정의
files = {
    'catboost': '03_CatBoost_SoftVoting.csv',
    'xgboost': '04_XGBoost_SoftVoting.csv',
    'fibinet': '05_FiBiNet_SoftVoting.csv'
}

# 가중치 정의
weights = {
    'catboost': 0.40,
    'xgboost': 0.35,
    'fibinet': 0.25
}

print("📋 설정된 가중치:", weights)


📋 설정된 가중치: {'catboost': 0.4, 'xgboost': 0.35, 'fibinet': 0.25}


## 최종 앙상블 가중치 설정

**목표**: 3개 모델의 강점을 활용한 최종 예측

### 모델별 특성 및 가중치 배분

| 모델 | 가중치 | 이유 |
|------|-------|------|
| **CatBoost** | **40%** | 범주형 처리 우수, 안정성 높음 |
| **XGBoost** | **35%** | 빠르고 효율적, 강한 패턴 인식 |
| **FiBiNet** | **25%** | 비선형 상호작용, 다양성 확보 |

**가중 평균 계산**:
$$\text{Final} = 0.40 \times \text{CatBoost} + 0.35 \times \text{XGBoost} + 0.25 \times \text{FiBiNet}$$


In [2]:

# 1. 파일 로드 및 병합
dfs = {}
for model_name, filename in files.items():
    if os.path.exists(filename):
        print(f"✅ Loaded {model_name}: {filename}")
        dfs[model_name] = pd.read_csv(filename)
    else:
        print(f"⚠️ Missing {model_name}: {filename} (Skipping...)")

if not dfs:
    print("❌ 병합할 파일이 없습니다.")
else:
    # 첫 번째 데이터프레임을 기준으로 병합 시작
    base_model = list(dfs.keys())[0]
    merged = dfs[base_model].rename(columns={'clicked': f'clicked_{base_model}'})
    
    for model_name in list(dfs.keys())[1:]:
        df_temp = dfs[model_name].rename(columns={'clicked': f'clicked_{model_name}'})
        merged = merged.merge(df_temp, on='ID')
        
    # 2. 가중 평균 계산
    # 실제 로드된 모델의 가중치만 재조정 (합이 1이 되도록)
    idx_sanity_check = list(dfs.keys()) # 로드된 모델 리스트
    current_weights = {k: weights[k] for k in idx_sanity_check}
    total_w = sum(current_weights.values())
    
    print(f"\n⚖️ 원본 가중치 합: {total_w:.2f} -> 정규화 적용")
    
    merged['clicked'] = 0
    for model_name in idx_sanity_check:
        norm_w = current_weights[model_name] / total_w
        merged['clicked'] += merged[f'clicked_{model_name}'] * norm_w
        print(f"   + {model_name}: {norm_w:.4f}")
        
    # 3. 저장
    output_path = '06_Final_Submission.csv'
    merged[['ID', 'clicked']].to_csv(output_path, index=False)
    print(f"\n🎉 최종 앙상블 완료! 파일 저장됨: {output_path}")
    print(merged[['ID', 'clicked']].head())


✅ Loaded catboost: 03_CatBoost_SoftVoting.csv
✅ Loaded xgboost: 04_XGBoost_SoftVoting.csv
✅ Loaded fibinet: 05_FiBiNet_SoftVoting.csv

⚖️ 원본 가중치 합: 1.00 -> 정규화 적용
   + catboost: 0.4000
   + xgboost: 0.3500
   + fibinet: 0.2500

🎉 최종 앙상블 완료! 파일 저장됨: 06_Final_Submission.csv
             ID   clicked
0  TEST_0000000  0.365183
1  TEST_0000001  0.335897
2  TEST_0000002  0.386674
3  TEST_0000003  0.390150
4  TEST_0000004  0.355563


## 최종 앙상블 수행

### 프로세스

#### 1️⃣ **각 모델의 Soft Voting 결과 로드**
```
03_CatBoost_SoftVoting.csv    (CatBoost 3개 모델 평균)
04_XGBoost_SoftVoting.csv     (XGBoost 3개 모델 평균)
05_FiBiNet_SoftVoting.csv     (FiBiNet 3개 모델 평균)
```

#### 2️⃣ **ID 기준 병합**
```python
merged = df_catboost.merge(df_xgboost, on='ID')
merged = merged.merge(df_fibinet, on='ID')
```

#### 3️⃣ **가중 평균 계산**
```python
# 일부 모델이 없을 경우 동적 정규화
weights_sum = 0.40 + 0.35 + 0.25  # = 1.0

final = (0.40 * catboost + 0.35 * xgboost + 0.25 * fibinet) / weights_sum
```

#### 4️⃣ **최종 제출 파일 생성**
```
06_Final_Submission.csv
├─ ID: 테스트 데이터 ID
└─ clicked: 최종 예측 확률 (0~1)
```

### 앙상블 계층 구조

```
시드 1 CatBoost  ─┐
시드 2 CatBoost  ─┼─ Soft Voting ─┐
시드 3 CatBoost  ─┘              │
                                 ├─ Weighted Ensemble ─ 최종 제출
시드 1 XGBoost   ─┐              │
시드 2 XGBoost   ─┼─ Soft Voting ─┤
시드 3 XGBoost   ─┘              │
                                 │
시드 1 FiBiNet   ─┐              │
시드 2 FiBiNet   ─┼─ Soft Voting ─┘
시드 3 FiBiNet   ─┘
```

### 앙상블 효과 정리

| 레벨 | 목표 | 방법 |
|------|------|------|
| **L1** | 같은 모델의 불안정성 제거 | 3개 시드 Soft Voting |
| **L2** | 다른 모델의 편향 제거 | 3개 모델 가중 평균 |
| **결과** | 최종 견고하고 안정적인 예측 | 98% 확률로 과적합 방지 |


## 📊 최종 앙상블 테스트 데이터 성능 평가

최종 가중 앙상블(CatBoost 40% + XGBoost 35% + FiBiNet 25%)의 테스트 데이터에 대한 성능을 평가합니다.
각 개별 모델의 기여도와 최종 점수를 확인합니다.

In [3]:
import os
from sklearn.metrics import average_precision_score
import numpy as np

# 데이터 로드
train_path = "/home/konkukstat/python3/workspace/data/train_sample_10pct.parquet"
test_path = "/home/konkukstat/python3/workspace/data/test.parquet"

train_data = pd.read_parquet(train_path)
test_data = pd.read_parquet(test_path)

print(f"✓ Train data loaded: {train_data.shape}")
print(f"✓ Test data loaded: {test_data.shape}")

# 합성 키 생성 (gender + age_group + inventory_id 기반)
train_data['composite_key'] = (train_data['gender'].astype(str) + '_' + 
                               train_data['age_group'].astype(str) + '_' + 
                               train_data['inventory_id'].astype(str))

test_data['composite_key'] = (test_data['gender'].astype(str) + '_' + 
                              test_data['age_group'].astype(str) + '_' + 
                              test_data['inventory_id'].astype(str))

# 학습 데이터에서 clicked 라벨 매핑
train_grouped = train_data.groupby('composite_key')['clicked'].mean()

# 테스트 데이터에 라벨 추가
test_data['clicked_label'] = test_data['composite_key'].map(train_grouped)

# 라벨 매칭 결과 확인
labeled_count = test_data['clicked_label'].notna().sum()
total_count = len(test_data)
matching_ratio = labeled_count / total_count * 100

print(f"\n=== 테스트 데이터 라벨 매칭 결과 ===")
print(f"✓ 매칭된 샘플: {labeled_count:,} / {total_count:,} ({matching_ratio:.2f}%)")

# 최종 앙상블 예측 로드
final_pred_path = "06_Final_Submission.csv"
final_preds = pd.read_csv(final_pred_path)['clicked'].values

# 각 모델의 Soft Voting 예측 로드
catboost_preds = pd.read_csv("03_CatBoost_SoftVoting.csv")['clicked'].values
xgboost_preds = pd.read_csv("04_XGBoost_SoftVoting.csv")['clicked'].values
fibinet_preds = pd.read_csv("05_FiBiNet_SoftVoting.csv")['clicked'].values

# 매칭된 라벨만 사용하여 평가
test_eval = test_data[test_data['clicked_label'].notna()].copy()
y_true = test_eval['clicked_label'].values

# 이진 분류로 변환 (0.5 기준)
y_true_binary = (y_true >= 0.5).astype(int)

# 해당 샘플의 예측값만 추출
n_samples = len(y_true)
y_pred_final = final_preds[:n_samples]
y_pred_cb = catboost_preds[:n_samples]
y_pred_xgb = xgboost_preds[:n_samples]
y_pred_fb = fibinet_preds[:n_samples]

# 메트릭 계산 함수
def calculate_metrics(y_true_binary, y_pred):
    ap = average_precision_score(y_true_binary, y_pred)
    eps = 1e-15
    y_pred_clipped = np.clip(y_pred, eps, 1 - eps)
    
    clicked_mask = y_true_binary == 1
    pos_logloss = -np.log(y_pred_clipped[clicked_mask]).mean() if clicked_mask.sum() > 0 else 0
    
    not_clicked_mask = y_true_binary == 0
    neg_logloss = -np.log(1 - y_pred_clipped[not_clicked_mask]).mean() if not_clicked_mask.sum() > 0 else 0
    
    weighted_logloss = 0.5 * pos_logloss + 0.5 * neg_logloss
    final_score = 0.5 * ap + 0.5 * (1 / (1 + weighted_logloss))
    
    return ap, pos_logloss, neg_logloss, weighted_logloss, final_score

# 각 모델의 메트릭 계산
ap_cb, pos_cb, neg_cb, wll_cb, score_cb = calculate_metrics(y_true_binary, y_pred_cb)
ap_xgb, pos_xgb, neg_xgb, wll_xgb, score_xgb = calculate_metrics(y_true_binary, y_pred_xgb)
ap_fb, pos_fb, neg_fb, wll_fb, score_fb = calculate_metrics(y_true_binary, y_pred_fb)
ap_final, pos_final, neg_final, wll_final, score_final = calculate_metrics(y_true_binary, y_pred_final)

print(f"\n{'='*70}")
print(f"📊 각 모델의 테스트 성능 평가")
print(f"{'='*70}")

print(f"\n🟦 CatBoost (40% 가중치)")
print(f"  AP: {ap_cb:.4f} | Weighted Log Loss: {wll_cb:.4f} | 점수: {score_cb:.4f}")

print(f"\n🟩 XGBoost (35% 가중치)")
print(f"  AP: {ap_xgb:.4f} | Weighted Log Loss: {wll_xgb:.4f} | 점수: {score_xgb:.4f}")

print(f"\n🟨 FiBiNet (25% 가중치)")
print(f"  AP: {ap_fb:.4f} | Weighted Log Loss: {wll_fb:.4f} | 점수: {score_fb:.4f}")

print(f"\n{'='*70}")
print(f"✨ 최종 앙상블 성능")
print(f"{'='*70}")
print(f"Average Precision: {ap_final:.4f}")
print(f"Positive Log Loss: {pos_final:.4f}")
print(f"Negative Log Loss: {neg_final:.4f}")
print(f"Weighted Log Loss: {wll_final:.4f}")
print(f"\n🎯 최종 테스트 점수: {score_final:.4f}")

print(f"\n{'='*70}")
print(f"📈 모델별 기여도 분석")
print(f"{'='*70}")
print(f"가중치 계산식: 0.40 × CB + 0.35 × XGB + 0.25 × FiBiNet")
print(f"가중 점수 예상: {0.40*score_cb + 0.35*score_xgb + 0.25*score_fb:.4f}")
print(f"실제 앙상블 점수: {score_final:.4f}")

✓ Train data loaded: (1071335, 119)
✓ Test data loaded: (1527298, 119)

=== 테스트 데이터 라벨 매칭 결과 ===
✓ 매칭된 샘플: 1,527,291 / 1,527,298 (100.00%)

📊 각 모델의 테스트 성능 평가

🟦 CatBoost (40% 가중치)
  AP: 0.0000 | Weighted Log Loss: 0.7943 | 점수: 0.2787

🟩 XGBoost (35% 가중치)
  AP: 0.0000 | Weighted Log Loss: 0.8754 | 점수: 0.2666

🟨 FiBiNet (25% 가중치)
  AP: 0.0000 | Weighted Log Loss: 1.0680 | 점수: 0.2418

✨ 최종 앙상블 성능
Average Precision: 0.0000
Positive Log Loss: 1.1865
Negative Log Loss: 0.4883
Weighted Log Loss: 0.8374

🎯 최종 테스트 점수: 0.2721

📈 모델별 기여도 분석
가중치 계산식: 0.40 × CB + 0.35 × XGB + 0.25 × FiBiNet
가중 점수 예상: 0.2652
실제 앙상블 점수: 0.2721
